In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets

from os import listdir
from os.path import isfile, join, exists

In [ ]:
def create_paper_summary_data_frame():
    doi = input("DOI for paper: ")
    directory = input("Data Directory for paper: ")
    #list of all the [folders/plots] for paper ()
    plots_in_dir = listdir(directory)  #jlu
    num_plots = len(plots_in_dir)
    
    for i in range(num_plots):
        paper_descript = input("One sentence summary of paper: ")   #   jlu
        plot_foldername = directory + "/" + plots_in_dir[i]
        # list of all the csv's associated with plot
        data_segs_for_plot = listdir(plot_foldername)  #jlu 
        print(data_segs_for_plot)
        num_segs_in_plot = len(data_segs_for_plot)
        x_label = widgets.Dropdown(
            options={'Air Vel [cm/s]': 1,'FSR [mm/s]': 2,'d_c [mm]': 3, 'd_i [mm]': 4, 'P_atm': 5, '[O2]': 6, 'g': 7, 
                     'q_ext [kW/m^2]': 8, 'core material': 9, 'insulation material': 10,'orientation (vert/horiz)': 11, 
                     'up | down': 12},
            value=1,
            description='x - axis:',
            disabled=False,
        )
        display(x_label)
        y_label = widgets.Dropdown(
            options={'Air Vel [cm/s]': 1,'FSR [mm/s]': 2,'d_c [mm]': 3, 'd_i [mm]': 4, 'P_atm': 5, '[O2]': 6, 'g': 7, 
                     'q_ext [kW/m^2]': 8, 'core material': 9, 'insulation material': 10,'orientation (vert/horiz)': 11, 
                     'up | down': 12},
            value=2,
            description='y - axis:',
            disabled=False,
        )
        display(y_label)
        cols = ['Air Vel [cm/s]','FSR [mm/s]','d_c [mm]', 'd_i [mm]',
                      'P_atm', '[O2]', 'g', 'q_ext [kW/m^2]', 'core material', 
                      'insulation material','orientation (vert/horiz)', 'up | down']
        df1 = pd.DataFrame(index = plots_in_dir, 
                           columns = cols)
        for j in range(num_segs_in_plot):
            name_of_seg_in_plot = plot_foldername + "/" + data_segs_for_plot[j]
            print(name_of_seg_in_plot)
            skeleton_row = []
            for k in cols:
                # we are also importing that from the plot digitized data
                skeleton_row.append(input("For data seg " + data_segs_for_plot[j] + " " + k))
            print(df1)

In [ ]:
create_paper_summary_data_frame()

In [ ]:
df_read1 = pd.read_csv('kobayashi\8mm_Cu.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read2 = pd.read_csv('kobayashi\9mm_Cu.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read3 = pd.read_csv('kobayashi\8mm_SS.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read4 = pd.read_csv('kobayashi\9mm_SS.csv', skiprows=[0,1,2,3,4], index_col=False)
df_read5 = pd.read_csv
df_togeter = pd.concat([df_read1, df_read2])
print(df_togeter)